<a href="https://colab.research.google.com/github/Farhanahoque251/Drug-Discovery-with-Python/blob/main/Lung_cancer_NSCLC_N6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install chembl_webresource_client rdkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
jsonschema 4.19.2 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.
referencing 0.32.0 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.


In [ ]:

# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [ ]:
# Target search for NSCLC
target = new_client.target
target_query = target.search('NSCLC')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,NSCLC,19.0,False,CHEMBL612554,[],CELL-LINE,9606
1,[],Homo sapiens,Lung NSCLC-N6 cell-line,12.0,False,CHEMBL614016,[],CELL-LINE,9606


In [ ]:
targets.shape

(2, 9)

In [ ]:
selected_target = targets.target_chembl_id[1]
selected_target

'CHEMBL614016'

In [ ]:
selected_target = {'target_chembl_id': 'CHEMBL614016	'}


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

# Get bioactivities for the target
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target['target_chembl_id'], pchembl_value__isnull=False).filter(standard_type="IC50")

# Collect unique compound IDs
compounds = {x['molecule_chembl_id'] for x in res}

# Fetch molecule details and apply Lipinski's Rule of Five
molecules = []

for compound_id in compounds:
    try:
        mol_data = new_client.molecule.filter(molecule_chembl_id=compound_id).only('molecule_structures')
        smiles = mol_data[0]['molecule_structures']['canonical_smiles']
        mol = Chem.MolFromSmiles(smiles)

        if mol:
            mw = Descriptors.MolWt(mol)
            hbd = Lipinski.NumHDonors(mol)
            hba = Lipinski.NumHAcceptors(mol)
            clogp = Chem.Crippen.MolLogP(mol)

            if mw <= 500 and hbd <= 5 and hba <= 10 and clogp <= 5:
                molecules.append({
                    'SMILES': smiles,
                    'MolecularWeight': mw,
                    'HBD': hbd,
                    'HBA': hba,
                    'CLogP': clogp
                })
    except Exception as e:
        print(f"Error processing compound {compound_id}: {e}")

# Print or process the filtered molecules
for molecule in molecules:
    print(molecules)


In [ ]:
print(len(molecules))

0
